In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

**Window Functions**

In [2]:
spark_session = SparkSession.builder.appName("AdvancedTransformations").getOrCreate()
df_spark = spark_session.read.csv("train.csv", header=True, inferSchema=True)

25/11/24 12:34:30 WARN Utils: Your hostname, Shithils-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.226 instead (on interface en0)
25/11/24 12:34:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/24 12:34:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
w=Window.partitionBy('Store').orderBy('Date')
df_spark.withColumn('Prev_Day_Sales', lag('Sales',1).over(w)).show(5)
df_spark.withColumn('Next_Day_Sales', lead('Sales',1).over(w)).show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Prev_Day_Sales|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|          NULL|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|             0|
|   28|        4|2013-01-03| 5287|      641|   1|    0|           0|            1|          4958|
|   28|        5|2013-01-04| 5481|      623|   1|    0|           0|            1|          5287|
|   28|        6|2013-01-05| 2070|      287|   1|    0|           0|            0|          5481|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+
only showing top 5 rows

+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------

In [4]:
df_spark.withColumn('Sales_Rank',rank().over(w.orderBy(col('Sales').desc()))).show()

+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Sales_Rank|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------+
|   28|        5|2014-08-15|18474|     1202|   1|    0|           0|            1|         1|
|   28|        5|2014-09-05|16611|     1592|   1|    1|           0|            0|         2|
|   28|        4|2014-08-14|13195|      936|   1|    0|           0|            1|         3|
|   28|        1|2013-12-02|12968|     1038|   1|    1|           0|            0|         4|
|   28|        1|2013-12-16|12958|     1089|   1|    1|           0|            0|         5|
|   28|        1|2014-08-18|12724|      978|   1|    1|           0|            1|         6|
|   28|        2|2014-01-07|12486|     1188|   1|    1|           0|            0|         7|
|   28|        1|2013-12-23|12373|     1251|   1|    0|     

In [5]:
df_spark.filter(col("Store") == 28).select("Store", "Sales").distinct().show(5)

+-----+-----+
|Store|Sales|
+-----+-----+
|   28| 1891|
|   28| 3543|
|   28| 3349|
|   28| 7301|
|   28| 7216|
+-----+-----+
only showing top 5 rows



In [6]:
df_spark.withColumn(
    "Row_Num",
    row_number().over(w.orderBy(col("Sales").desc()))
).show(5)


+-----+---------+----------+-----+---------+----+-----+------------+-------------+-------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Row_Num|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+-------+
|   28|        5|2014-08-15|18474|     1202|   1|    0|           0|            1|      1|
|   28|        5|2014-09-05|16611|     1592|   1|    1|           0|            0|      2|
|   28|        4|2014-08-14|13195|      936|   1|    0|           0|            1|      3|
|   28|        1|2013-12-02|12968|     1038|   1|    1|           0|            0|      4|
|   28|        1|2013-12-16|12958|     1089|   1|    1|           0|            0|      5|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+-------+
only showing top 5 rows



In [8]:
df_spark.withColumn('Running_Avg_Sales',avg('Sales').over(w)).show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Running_Avg_Sales|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|              0.0|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|           2479.0|
|   28|        4|2013-01-03| 5287|      641|   1|    0|           0|            1|           3415.0|
|   28|        5|2013-01-04| 5481|      623|   1|    0|           0|            1|           3931.5|
|   28|        6|2013-01-05| 2070|      287|   1|    0|           0|            0|           3559.2|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+-----------------+
only showing top 5 rows



Sales Anomaly Detection

In [18]:
df_spark.withColumn('Sales_Diff', col('Sales')-lag('Sales',1).over(w)).withColumn('Sales_Pct_Change', col('Sales_Diff')/lag('Sales',1).over(w)).show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------+--------------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Sales_Diff|    Sales_Pct_Change|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+----------+--------------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|      NULL|                NULL|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|      4958|                NULL|
|   28|        4|2013-01-03| 5287|      641|   1|    0|           0|            1|       329|  0.0663574021782977|
|   28|        5|2013-01-04| 5481|      623|   1|    0|           0|            1|       194|0.036693777189332324|
|   28|        6|2013-01-05| 2070|      287|   1|    0|           0|            0|     -3411| -0.6223316912972086|
+-----+---------+----------+-----+---------+----+-----+------------+------------

25/11/24 16:10:23 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1268653 ms exceeds timeout 120000 ms
25/11/24 16:10:23 WARN SparkContext: Killing executors is not supported by current scheduler.
25/11/24 16:10:24 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [20]:
spark_session.stop()